# Load env

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import importlib
from reports_etl import *
from enrich_holdings import *
from fossil_classification import *
from holdings_analysis import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [42]:
last_updated()

.py file last modified: 2023-07-06 00:23:44.008761


# Bulk download reports
Using the http response recorded while searching for reports here:
https://employersinfocmp.cma.gov.il/#/publicreports

In [5]:
# deprecated - moved to using response directly

# get reports from response.json search results
# reports_path = "data/downloaded reports/company reports/2022Q4/"
# reports_path = "data/downloaded reports/fund reports/2020Q1/"
# reports = get_reports_from_response(reports_path)

## getting response directly from the reports website

In [6]:
from_year = '2023'
from_q = '1'

# Fund Reports

In [7]:
reports_path = "data/downloaded reports/fund reports/" + from_year + "Q" + from_q + "/"
# bituach, gemel, pension
report_types = ['71100072', '71100076', '71100075']

# Company Reports

In [10]:
reports_path = "data/downloaded reports/company reports/" + from_year + "Q" + from_q + "/"
# bituach, gemel, pension
report_types = ['71100071', '71100073', '71100074']

In [11]:
reports = pd.DataFrame()
for rt in report_types:
    add_reports = get_report_data_into_data_frame(
        from_year = from_year,
        from_q = from_q,
        to_year = from_year,
        to_q = from_q,
        report_type = rt,
        system = "",
        )
    reports = pd.concat([reports, add_reports], axis=0, ignore_index=True)

number of reports for 2023 q1 until 2023 q1: 8
number of reports for 2023 q1 until 2023 q1: 47
number of reports for 2023 q1 until 2023 q1: 11


In [12]:
reports.loc[reports["SystemName"] == 'חיים ואובדן כושר עבודה', "SystemName"] = 'ביטוח'

In [14]:
reports.to_csv(reports_path + "reports.csv", index=False)

In [15]:
reports.groupby("SystemName").size()

SystemName
ביטוח     8
גמל      47
פנסיה    11
dtype: int64

## Check for missing fund reports

In [ ]:
json_keyfile_name = '../keys/israeli-fossil-holdings-db-03b783f17e5f.json'

gc = connect_to_gspreadsheets_api(json_keyfile_name)

In [ ]:
gss_url = "https://docs.google.com/spreadsheets/d/1XyJtEGtlFNsbIzCOHnfakiFOjtXXrn4jobHeu79IbRc"
add_document_ids_to_all_sheets(gc, gss_url, reports)

In [ ]:
system_id_cols= {
        "פנסיה": ['NUM_HEVRA','ID_MASLUL_RISHUY'],
        "גמל": ['NUM_HEVRA','ID'],
        "ביטוח": ['NUM_HEVRA','ID']
    }

sys = "גמל"

ws = gss.worksheet(sys)
df = pd.DataFrame(ws.get_all_records())
df_with_document_id = add_document_id_by_cols(df, reports, sys, system_id_cols[sys])
df_with_document_id = df_with_document_id.astype(str)
ws.update([df_with_document_id.columns.values.tolist()] + df_with_document_id.values.tolist())

In [ ]:
df_with_document_id.head()

In [ ]:
add_document_ids_to_all_sheets(gc, gss_url, reports)

In [ ]:
# reports[
#     reports["DocumentId"] == 2308537
# ]
# reports.head()

In [ ]:
# filter specific reports for quick analysis
# reports = reports[reports["ParentCorpName"].str.startswith("מגדל")]

In [ ]:
reports[
    reports["ProductNum"].isin(reports.groupby("ProductNum").size().loc[lambda x: x>1].index)
]

## download reports

In [16]:
# wrap with try, loop until no more files left
import glob
from pathlib import Path

# download missing files
downloaded = [Path(path).stem for path in glob.glob(reports_path + "*.xls*")]
print("already downloaded {} out of {}".format(len(downloaded), len(reports)))

reports_to_download = reports[
    ~reports["DocumentId"].astype('str').isin(downloaded)
]

len(reports_to_download)

already downloaded 0 out of 66


66

In [17]:
# download reports - run only once per response
download_reports(reports_to_download, reports_path, sleep=3)

# Process multiple reports -> unified file

In [18]:
reports_fn_list = get_filename_list(reports_path)

number of files to be pre-processed: 66


## Pre-process - check sheet names and column names, add to standardization if needed

In [19]:
pre_process_reports(reports_fn_list)

/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


{'סכום נכסי הקרן': 66, 'מזומנים': 66, 'תעודות התחייבות ממשלתיות': 66, 'תעודות חוב מסחריות': 66, 'אג"ח קונצרני': 66, 'מניות': 66, 'קרנות סל': 66, 'קרנות נאמנות': 66, 'כתבי אופציה': 66, 'אופציות': 66, 'חוזים עתידיים': 66, 'מוצרים מובנים': 66, 'לא סחיר - תעודות התחייבות ממשלתי': 66, 'לא סחיר - תעודות חוב מסחריות': 66, 'לא סחיר - אג"ח קונצרני': 66, 'לא סחיר - מניות': 66, 'לא סחיר - קרנות השקעה': 66, 'לא סחיר - כתבי אופציה': 66, 'לא סחיר - אופציות': 66, 'לא סחיר - חוזים עתידיים': 66, 'לא סחיר - מוצרים מובנים': 66, 'הלוואות': 66, 'פקדונות מעל 3 חודשים': 66, 'זכויות מקרקעין': 66, 'השקעה בחברות מוחזקות': 66, 'השקעות אחרות': 66, 'יתרת התחייבות להשקעה': 66, 'עלות מתואמת אג"ח קונצרני סחיר': 66, 'עלות מתואמת אג"ח קונצרני ל.סחיר': 66, 'עלות מתואמת מסגרות אשראי ללווים': 66, '{PL}PickLst': 3}


/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,מזומנים,תעודות התחייבות ממשלתיות,תעודות חוב מסחריות,"אג""ח קונצרני",מניות,קרנות סל,קרנות נאמנות,כתבי אופציה,אופציות,חוזים עתידיים,מוצרים מובנים,לא סחיר - תעודות התחייבות ממשלתי,לא סחיר - תעודות חוב מסחריות,"לא סחיר - אג""ח קונצרני",לא סחיר - מניות,לא סחיר - קרנות השקעה,לא סחיר - כתבי אופציה,לא סחיר - אופציות,לא סחיר - חוזים עתידיים,לא סחיר - מוצרים מובנים,הלוואות,פקדונות מעל 3 חודשים,זכויות מקרקעין,השקעה בחברות מוחזקות,השקעות אחרות,"עלות מתואמת אג""ח קונצרני סחיר","עלות מתואמת אג""ח קונצרני ל.סחיר",עלות מתואמת מסגרות אשראי ללווים,{PL}PickLst
"תא ללא תוכן, המשך בתא הבא",5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,NaN
שם המנפיק/שם נייר ערך,66.00,66.00,66.00,66.00,66.00,66.00,64.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,62.00,66.00,66.00,66.00,66.00,66.00,NaN
"מספר ני""ע",66.00,66.00,66.00,66.00,66.00,66.00,64.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,NaN,NaN,66.00,66.00,66.00,66.00,NaN
מספר מנפיק,66.00,NaN,66.00,66.00,66.00,66.00,64.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,NaN,66.00,NaN,NaN,NaN,NaN,NaN
דירוג,66.00,66.00,66.00,66.00,NaN,NaN,64.00,NaN,NaN,NaN,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,NaN,66.00,66.00,66.00,66.00,66.00,NaN
שם מדרג,66.00,66.00,66.00,66.00,NaN,NaN,64.00,NaN,NaN,NaN,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,NaN,66.00,66.00,66.00,66.00,66.00,NaN
סוג מטבע,66.00,66.00,66.00,66.00,66.00,66.00,64.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,62.00,66.00,66.00,66.00,66.00,66.00,NaN
שעור ריבית,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,NaN,66.00,66.00,66.00,66.00,66.00,NaN
תשואה לפדיון,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,66.00,NaN,NaN,NaN,NaN,NaN,66.00,66.00,66.00,NaN,66.00,66.00,NaN,NaN,NaN,NaN
שווי,66.00,66.00,66.00,66.00,66.00,66.00,64.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,66.00,62.00,66.00,66.00,66.00,66.00,66.00,NaN


## Debugging problematic reports

In [20]:
# fn = reports_path + "2391836.xlsx"
# fn = "/Users/urimarom/Downloads/513621110_psum_p422_1.xlsx"
# report = pd.read_excel(fn, sheet_name=2)

# from xlrd import open_workbook, XLRDError
# book = open_workbook(fn)

In [21]:
# TODO: look into reports with errors. some excel files get 'TypeError'
# fn = reports_path + "2287555.xlsx"
# report = pd.read_excel(fn)

# process holding reports
1. summary sheets (for verification)
2. holding sheets - raw data

## 1. summary sheets

In [22]:
all_summary_sheets = process_summary_sheets(reports_fn_list)

/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:405: FutureWarning: The default value of regex will change from True to False in a future version.
  all_summary_sheets["pct_num"] = all_summary_sheets["pct"].astype(str).str.replace(r'[\%\s-]', '')


In [23]:
totals = get_totals(all_summary_sheets)

Number of totals found: 66


In [24]:
totals["sum_num"].sum()

1773415064.4319067

## 2. Holdings

In [43]:
# extract all holdings from reports
all_holdings_clean = clean_holdings(extract_holdings(reports_fn_list))

/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing report 66 out of 66
before cleaning: 99172
 after cleaning: 86252


# add fund and company data to holdings

In [44]:
## Manually add missing corps to reports - only if needed, for funds processing
missing_corps = pd.DataFrame(columns=reports.columns)
missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
                                      'ParentCorpName': 'מנורה מבטחים ביטוח בע"מ',
                                      'ParentCorpLegalId': '520042540',
                                      'ReportPeriodDesc': ''
                                     }, ignore_index=True)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'כלל חברה לביטוח בע"מ',
#                                       'ParentCorpLegalId': '520024647',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)

# add infinity
# missing_corps = missing_corps.append({'SystemName': 'גמל',
#                                       'ParentCorpName': 'אינפיניטי השתלמות, גמל ופנסיה בע"מ',
#                                       'ParentCorpLegalId': '513621110',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)

missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
                                      'ParentCorpName': 'הפניקס חברה לביטוח בע"מ',
                                      'ParentCorpLegalId': '520023185',
                                      'ReportPeriodDesc': ''
                                     }, ignore_index=True)

reports = reports.append(missing_corps)

/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_22533/3239055288.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_22533/3239055288.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_22533/3239055288.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reports = reports.append(missing_corps)


In [89]:
all_holdings_final = add_report_data(all_holdings_clean, reports)

/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


# Compare totals from summary sheets with totals from holdings

In [98]:
print("total from summary sheets: {:,.2f}".format(totals["sum_num"].sum()))
print("total from holdings sheets: {:,.2f}".format(all_holdings_final["שווי"].sum()))
print("diff: {:,.2f}".format(totals["sum_num"].sum() - all_holdings_final["שווי"].sum()))

total from summary sheets: 1,773,415,064.43
total from holdings sheets: 1,769,822,795.14
diff: 3,592,269.29


In [99]:
totals_from_holdings = all_holdings_final.groupby(["report_id", "ParentCorpName"])[["שווי"]].sum().reset_index()
totals_from_holdings.index = totals_from_holdings.index.astype('str')

In [100]:
totals_comparison = totals_from_holdings.merge(totals,
                                               on='report_id',
                                               how='outer'
                                              )

totals_comparison["diff"] = totals_comparison["sum_num"] - totals_comparison["שווי"]
totals_comparison["diff_pct"] = abs(totals_comparison["diff"] * 100.00 /  totals_comparison["שווי"])
totals_comparison[abs(totals_comparison["diff"]) > 1].sort_values("diff_pct", ascending=False)

,report_id,ParentCorpName,שווי,asset,sum,pct,pct_num,sum_num,diff,diff_pct
3,2446084,"חברת הגמל לעובדי האוניברסיטה העברית בע""מ","2,908,753.74","סה""כ סכום נכסי המסלול או הקרן","2,509,101.52",1,1,"2,509,101.52","-399,652.22",13.74
19,2459609,"קרן החסכון לצבא הקבע - חברה לניהול קופות גמל בע""מ","5,457,325.92","סה""כ סכום נכסי המסלול או הקרן","5,816,382.13",1,1,"5,816,382.13","359,056.21",6.58
47,2465009,"הראל חברה לביטוח בע""מ","70,102,485.99","סה""כ סכום נכסי המסלול או הקרן","72,102,607.45",1,1,"72,102,607.45","2,000,121.46",2.85
20,2459658,"הראל פנסיה וגמל בע""מ","50,876,223.68","סה""כ סכום נכסי המסלול או הקרן","52,137,294.39",1,1,"52,137,294.39","1,261,070.71",2.48
49,2465220,"מחוג - מינהל גמל לעובדי חברת חשמל לישראל בע""מ","2,225,088.70","סה""כ סכום נכסי המסלול או הקרן","2,233,611.74",1,1,"2,233,611.74","8,523.04",0.38
9,2449311,"קופת""ג של עובדי עירית חיפה","221,324.22",סה''כ סכום נכסי המסלול או הקרן,"220,524.01",1,1,"220,524.01",-800.21,0.36
27,2460563,החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויו...,"3,939,314.55",סה''כ סכום נכסי המסלול או הקרן,"3,925,976.74",1,1,"3,925,976.74","-13,337.81",0.34
45,2464784,"הראל פנסיה וגמל בע""מ","131,464,713.35","סה""כ סכום נכסי המסלול או הקרן","131,841,294.06",1,1,"131,841,294.06","376,580.70",0.29
48,2465036,"לעתיד חברה לניהול קרנות פנסיה בע""מ","1,300,051.82","סה""כ סכום נכסי המסלול או הקרן","1,300,356.54",1,1,"1,300,356.54",304.72,0.02
10,2449390,קופת תגמולים של עובדי אל על נתיבי אוויר לישראל...,"749,708.35",סה''כ סכום נכסי המסלול או הקרן,"749,696.26",1,1,"749,696.26",-12.09,0.00


In [93]:
# TODO: add comparison by holding type (summary vs. sheets)
# report drill down
report_id = '2465009'
def report_sums_by_type(report_id):
    rep = all_holdings_final[all_holdings_final["report_id"] == report_id]
    print("total sum by holdings: {:,.2f}".format(rep["שווי"].sum()))
    return pd.DataFrame(rep.groupby("holding_type")["שווי"].agg('sum').map('{:,.2f}'.format))
    
report_sums_by_type(report_id)

total sum by holdings: 70,102,485.99


,שווי
holding_type,
"אג""ח קונצרני","6,724,011.50"
אופציות,-192.02
הלוואות,"10,065,488.95"
השקעה בחברות מוחזקות,0.00
השקעות אחרות,"417,255.58"
זכויות מקרקעין,"1,817,468.75"
חוזים עתידיים,"526,056.54"
כתבי אופציה,"2,977.18"
"לא סחיר - אג""ח קונצרני","587,060.06"


In [94]:
# trying to clean asset type so summary sheets can be joined with holding totals (clean to holding type)

# def clean_asset_type(summary_sheets, report_id):
#     summary_sheets = summary_sheets[summary_sheets["report_id"] == report_id]
#     nan_locs = (summary_sheets["sum"].isnull())
#     non_tradeable_heading_loc = summary_sheets["asset"].str.contains("לא סחיר").idxmax()
#     tradable = (~summary_sheets["asset"].str.contains("לא סחיר")) & (summary_sheets["asset"].str.contains("סחיר"))
#     tradable_heading_loc = tradable.idxmax()
#     tradable_min_loc = tradable_heading_loc + 1
#     tradable_max_loc = nan_locs.iloc[tradable_min_loc:].idxmax() - 1
#     print("tradable_min_loc: {}, tradable_max_loc: {}").format(non_tradeable_heading_loc, non_tradeable_heading_loc)
#     return non_tradeable_heading_loc
# # summary_sheets.iloc[tradable_min_loc:tradable_max_loc], summary_sheets.iloc[tradable_heading_loc]
    


# clean_asset_type(all_summary_sheets, report_id)

In [95]:
all_summary_sheets[
    all_summary_sheets["report_id"] == report_id
]

,asset,sum,pct,report_id,pct_num,sum_num
0,א. מזומנים,"5,888,175.16",0.08,2465009,0.08170959869917406,"5,888,175.16"
1,ב. ניירות ערך סחירים:,"39,430,811.03",0.55,2465009,0.5466294332140752,"39,430,811.03"
2,(1) תעודות התחייבות ממשלתיות,"12,705,360.21",0.18,2465009,0.17631097160749398,"12,705,360.21"
3,(2) תעודות חוב מסחריות,0,0,2465009,0,0
4,"(3) אג""ח קונצרני","6,724,011.50",0.09,2465009,0.093074509084867,"6,724,011.50"
5,(4) מניות,"9,072,279.62",0.13,2465009,0.12559066702359123,"9,072,279.62"
6,קרנות סל (5),"8,199,752.44",0.11,2465009,0.11377758279342025,"8,199,752.44"
7,תעודות השתתפות בקרנות נאמנות (6),"2,156,550.32",0.03,2465009,0.02992622616471131,"2,156,550.32"
8,כתבי אופציה (7),"2,977.18",0.00,2465009,4.13140822418752e05,"2,977.18"
9,אופציות (8),-192.02,-0.00,2465009,2.6646720942533017e06,-192.02


### totals discrepancies
<u>2021Q2</u><br>
* In report 2128339 - there's a redundant line in נדלן, consider removing manually when all files for Q3 are here - removing
* In report 2153994 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)
* In report 2146293 - the value of holdings in לא סחיר - אג"ח קונצרני are way off. leaving as is for now.

<u>2021Q3</u>
* 2148642 - missing exactly 10000 in מזומנים. leaving as is.
* 2148994 - dropped a line in הלוואות withno security number. not fixing for now.

<u>2021Q3</u>
* 2222108 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)

<u>2022Q1</u>
* 2235319 - נדלן. לא תוקן (לא משתתף במדרג)
* 2260140 - הפניקס פנסיה. הבדלים במזומנים. משאיר ככה כרגע.
* 2256493 - missing one holding in mutual funds, 50M

<u>2022Q2</u>
* 2308537 - הפניקס פנסיה. הבדלים במזומנים. משאיר ככה כרגע.

<u>2022Q4</u>
* 2409834 - הפניקס פנסיה - הבדלים במזומנים. הוספתי שורה לתיקון.

<u>2023Q1</u>
* 2468588, 2469774 - הפניקס - הבדלים במזומנים. **לא** הוספתי שורה לתיקון.



## Removing redundant rows

In [96]:
# remove_rows = (
#     (all_holdings_final["report_id"] == '2128339') &
#     (all_holdings_final["holding_type"] == 'זכויות מקרקעין') &
#     (all_holdings_final["שעור מנכסי אפיק ההשקעה"] == 1)
# )

all_holdings_final = all_holdings_final.drop(all_holdings_final[remove_rows].index, axis=0)

# all_holdings_final.groupby("report_id").filter(
#     lambda x: x["שעור מנכסי אפיק ההשקעה"].sum() > 1
# ).head()

NameError: name 'remove_rows' is not defined

## Adding missing rows

## add rows for missing cash by total sheet

In [97]:
# adding a row for report 2153994, to compensate for missing holding values
report_ids_cash_fix = ['2468588', '2469774']

def cash_sum_fix_row(report_id):
    total_from_summary = all_summary_sheets[
        (all_summary_sheets["report_id"] == report_id) &
        (all_summary_sheets["asset"]).str.contains('מזומנים')
    ]["sum_num"]

    holdings_to_fix = all_holdings_final[
        (all_holdings_final["report_id"] == report_id) &
        (all_holdings_final["holding_type"] == 'מזומנים')
    ]

    total_from_holdings = holdings_to_fix["שווי"].sum()

    # calculate sum and pct to be added
    to_be_added_sum = float(total_from_summary) - float(total_from_holdings)
    to_be_added_pct = 1 - holdings_to_fix["שעור מנכסי אפיק ההשקעה"].sum()
    to_be_added_pct_of_report = float(to_be_added_sum) / float(totals[totals["report_id"] == report_id]["sum_num"])

    # prepare new row - take the highest amount row as base and edit it
    new_row = holdings_to_fix.sort_values("שווי", ascending=False).head(1)
    new_row["שם המנפיק/שם נייר ערך"] = "unknown - added to match sum"
    new_row['שווי'] = to_be_added_sum
    new_row['שעור מנכסי אפיק ההשקעה'] = to_be_added_pct
    new_row['שעור מסך נכסי השקעה'] = to_be_added_pct_of_report

    for col in ['מספר ני"ע', 'מספר מנפיק', 'דירוג', 'שם מדרג', 'סוג מטבע']:
        new_row[col] = None

    print("adding row: ")
    print(new_row)
    # adding the artificial row
    return new_row

for report_id in report_ids_cash_fix:
    all_holdings_final = all_holdings_final.append(cash_sum_fix_row(report_id))

adding row: 
              שם המנפיק/שם נייר ערך מספר ני"ע מספר מנפיק דירוג שם מדרג  \
77444  unknown - added to match sum      None       None  None    None   

      סוג מטבע שעור ריבית תשואה לפדיון          שווי  שעור מנכסי אפיק ההשקעה  \
77444     None          0            0 13,358,849.63                    0.82   

       שעור מסך נכסי השקעה report_id holding_type זירת מסחר תאריך רכישה מח"מ  \
77444                 0.17   2468588      מזומנים       NaN         NaN  NaN   

      ערך נקוב  שער פדיון/ריבית/דיבידנד לקבל שעור מערך נקוב מונפק ספק מידע  \
77444      NaN  NaN                      NaN                  NaN      NaN   

      ענף מסחר נכס בסיס קונסורציום כן/לא תאריך שערוך אחרון אופי הנכס  \
77444      NaN      NaN              NaN               NaN       NaN   

      שעור תשואה במהלך התקופה כתובת הנכס ריבית אפקטיבית SystemName  \
77444                     NaN        NaN            NaN      ביטוח   

                ParentCorpName ParentCorpLegalId ProductNum  Name ShortNa

/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_22533/1066877939.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_holdings_final = all_holdings_final.append(cash_sum_fix_row(report_id))
/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_22533/1066877939.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_holdings_final = all_holdings_final.append(cash_sum_fix_row(report_id))


In [101]:
# all_holdings_final[
#     (all_holdings_final["ParentCorpName"].str.startswith("אינפיניטי")) &
#     (all_holdings_final["SystemName"] == 'גמל')
# ].head()

all_holdings_final.groupby("report_id")[["שווי"]].sum().reset_index()

,report_id,שווי
0,2442471,"81,115.53"
1,2443214,"545,848.01"
2,2445861,"4,698,630.11"
3,2446084,"2,908,753.74"
4,2446230,"2,489,137.05"
5,2446594,"625,055.14"
6,2446755,"1,879,427.06"
7,2447804,"35,798.90"
8,2447869,"1,692,270.45"
9,2449311,"221,324.22"


In [102]:
all_holdings_final.to_csv(reports_path+"all_holdings.csv", index=False)

In [103]:
### uncomment when running on fund reports ###
# all_holdings_ever = all_holdings_final

## Export file for classification

In [104]:
cols_for_cls = [
    'שם המנפיק/שם נייר ערך', 'מספר ני"ע', 'מספר מנפיק',
    'שווי' ,'שעור מנכסי אפיק ההשקעה', 'שעור מסך נכסי השקעה', 
       'holding_type', 'זירת מסחר', 'תאריך רכישה', 'ערך נקוב', 'שער','שעור מערך נקוב מונפק',
       'ענף מסחר',
       'SystemName', 'ParentCorpName', 'ReportPeriodDesc'
]

holdings_for_cls = all_holdings_final[
    all_holdings_final["holding_type"].isin(['מניות', 'אג"ח קונצרני'])
]

holdings_for_cls[cols_for_cls].to_csv(reports_path+"holdings_for_cls.csv", index=False)

In [ ]:
# holdings_for_cls[
#     (holdings_for_cls['מספר ני"ע'].astype(str) == '11616780')
# ]

## add new processed holdings to all_holdings file

In [105]:
fetch_all_holdings_path()

'data/downloaded reports/company reports/all_holdings.csv'

In [106]:
# reload results from file
all_holdings_path = fetch_all_holdings_path()
new_holdings_path = reports_path+"all_holdings.csv"
all_holdings_ever = concat_from_csv_by_path(all_holdings_path, new_holdings_path)

pd.DataFrame(all_holdings_ever.groupby(['ReportPeriodDesc'])['שווי'].agg('sum').map('{:,.2f}'.format))

/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:690: DtypeWarning: Columns (6,7,13,20,22,23,24,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  all_holdings = pd.read_csv(all_holdings_path, dtype=holdings_dtypes())
/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:691: DtypeWarning: Columns (7,9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  new_holdings = pd.read_csv(new_holdings_path, dtype=holdings_dtypes())


,שווי
ReportPeriodDesc,
2020 רבעון 1,"1,222,418,614.88"
2020 רבעון 2,"1,307,193,119.07"
2020 רבעון 3,"1,373,544,876.41"
2020 רבעון 4,"1,461,378,236.82"
2021 רבעון 1,"1,548,168,300.50"
2021 רבעון 2,"1,593,940,074.16"
2021 רבעון 3,"1,640,889,505.45"
2021 רבעון 4,"1,776,286,225.46"
2022 רבעון 1,"1,778,911,368.39"


In [115]:
# update all holdings file - only for companies! - handle with care
# TODO: add backup process
all_holdings_ever.to_csv(fetch_all_holdings_path(), index=False)

<h1><center>*** END OF PART 1 ***</center></h1>

# Validate holdings file

In [111]:
q_year = from_year + ' רבעון ' + from_q
print(q_year)
all_holdings_curr_q = all_holdings_ever[all_holdings_ever["ReportPeriodDesc"] == q_year]
# pd.DataFrame(all_holdings_curr_q.groupby(['SystemName', 'ParentCorpName', 'report_id'])['שווי'].agg('sum').map('{:,.2f}'.format))
pd.DataFrame(all_holdings_curr_q.groupby(['ParentCorpName'])['שווי'].agg('sum').map('{:,.2f}'.format))

2023 רבעון 1


,שווי
ParentCorpName,
"איי. די. איי. חברה לביטוח בע""מ","293,098.93"
"איילון חברה לביטוח בע""מ","4,579,448.79"
"אינפיניטי השתלמות, גמל ופנסיה בע""מ","1,728,069.35"
"אלטשולר שחם גמל ופנסיה בע""מ","162,676,894.00"
"אנליסט קופות גמל בע""מ","21,109,327.68"
ארם גמולים - חברה לניהול קופות גמל בע''מ,"1,041,070.33"
"גל -ניהול קופות גמל לעובדי הוראה בע""מ","4,295,582.84"
"החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובדי המנהל והשירותים בע""מ","823,011.07"
"החברה המנהלת של קרן השתלמות של עובדי חברת החשמל לישראל בע""מ","625,055.14"


## Look for missing company reports

In [108]:
reports_cnt = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId'])['ReportPeriodDesc'].nunique().reset_index())
period_cnt = all_holdings_ever["ReportPeriodDesc"].nunique()

companies_with_missing_reports = reports_cnt[reports_cnt["ReportPeriodDesc"] < period_cnt]

In [114]:
available_reports_periods = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId', 'ReportPeriodDesc'])['שווי'].sum()).reset_index()
companies_w_missing_reports_available_reports = pd.merge(
    companies_with_missing_reports[['ParentCorpLegalId', 'SystemName']],
    available_reports_periods,
    on = ['ParentCorpLegalId', 'SystemName'],
    how="inner"
)
companies_w_missing_reports_available_reports.sort_values(['ParentCorpName', 'SystemName', 'ReportPeriodDesc'])

,ParentCorpLegalId,SystemName,ParentCorpName,ReportPeriodDesc,שווי
4,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2020 רבעון 1,"3,855,913.06"
5,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2020 רבעון 2,"4,093,800.67"
6,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2020 רבעון 3,"4,219,780.06"
7,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2020 רבעון 4,"4,484,911.89"
8,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2021 רבעון 1,"4,641,924.58"
9,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2021 רבעון 2,"4,827,252.48"
10,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2021 רבעון 3,"4,901,670.90"
11,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2021 רבעון 4,"5,105,081.47"
12,520042169,ביטוח,"איילון חברה לביטוח בע""מ",2022 רבעון 1,"4,979,113.30"
0,520030677,ביטוח,"איילון חברה לביטוח בע""מ",2022 רבעון 2,"4,675,172.84"


Located missing reports for major companies, manually downloaded

## Look for missing fund reports

### 1. Using company reports

In [ ]:
quarter = '2022Q1'
funds_path = "data/downloaded reports/fund reports " + quarter + "/all_holdings.csv"
company_path = "data/downloaded reports/company reports " + quarter + "/all_holdings.csv"

id_dtypes={"report_id":str, "ParentCorpLegalId":str, "ProductNum":str}

funds_holdings = pd.read_csv(funds_path, dtype=id_dtypes)
# change חיים ואובדן כושר עבודה to ביטוח
funds_holdings["SystemName"] = funds_holdings["SystemName"].str.replace("חיים ואובדן כושר עבודה", "ביטוח")
company_holdings = pd.read_csv(company_path, dtype=id_dtypes)

In [ ]:
funds_agg = funds_holdings.groupby(["ParentCorpName", "ParentCorpLegalId", "SystemName"])[["שווי"]].sum().reset_index()
company_agg = company_holdings.groupby(["ParentCorpName", "ParentCorpLegalId", "SystemName"])[["שווי"]].sum().reset_index()

comparison = pd.merge(left=company_agg,
                      right=funds_agg,
                      how='outer',
                      on=["ParentCorpLegalId", "SystemName"],
                      suffixes=['', '_funds']
                     )
comparison["ParentCorpName"] = comparison["ParentCorpName"].fillna(comparison["ParentCorpName_funds"])
comparison.drop("ParentCorpName_funds", axis=1, inplace=True)
comparison["sum_diff"] = comparison["שווי"] - comparison["שווי_funds"]

companies_missing_funds = comparison[(abs(comparison["sum_diff"]) > 1) |
                                     (comparison["sum_diff"].isna())
                                    ].sort_values("sum_diff", ascending=False)
companies_missing_funds

In [ ]:
funds_agg_by_product = funds_holdings.groupby([
    "ParentCorpName",
    "SystemName",
    "ProductNum",
    "ShortName",
    "report_id",
    "ParentCorpLegalId"
], dropna=False)[["שווי"]].sum().reset_index()

pd.merge(left=companies_missing_funds,
         right=funds_agg_by_product,
         how='left',
         on=['ParentCorpLegalId', "SystemName"]
        )
# .sort_values(["sum_diff", "ProductNum"], ascending=False)

### 2. using pensia-net, bituach-net, gemel-net
API calls to x-net gov site:
https://data.gov.il/dataset/pensia-net


In [ ]:
import urllib.request as urlreq

def x_net_types():
    return {
    "DEPOSITS": float,
    "WITHDRAWLS": float,
    "INTERNAL_TRANSFERS": float,
    "NET_MONTHLY_DEPOSITS": float,
    "TOTAL_ASSETS": float,
    "AVG_ANNUAL_MANAGEMENT_FEE": float,
    "AVG_DEPOSIT_FEE": float,
    "MONTHLY_YIELD": float,
    "YEAR_TO_DATE_YIELD": float,
    "ACTUARIAL_ADJUSTMENT": float,
    "YIELD_TRAILING_3_YRS": float, 
    "YIELD_TRAILING_5_YRS": float,
    "AVG_ANNUAL_YIELD_TRAILING_3YRS": float,
    "AVG_ANNUAL_YIELD_TRAILING_5YRS": float,
    "STANDARD_DEVIATION": float,
    "ALPHA": float,
    "SHARPE_RATIO": float,
    "LIQUID_ASSETS_PERCENT": float,
    "STOCK_MARKET_EXPOSURE": float,
    "FOREIGN_EXPOSURE": float,
    "FOREIGN_CURRENCY_EXPOSURE": float
}

def read_from_x_net(resource_id):
    url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=' + resource_id + '&limit=32000'
    fileobj = urlreq.urlopen(url)
    df = pd.DataFrame(pd.read_json(fileobj)["result"]["records"])
    # handle dtypes
    for c in x_net_types():
        if c in df:
            df[c] = df[c].astype(x_net_types()[c], errors='ignore')
    return df

pensia_net = read_from_x_net('6d47d6b5-cb08-488b-b333-f1e717b1e1bd')
pensia_net['SystemName'] = 'פנסיה'
bituach_net = read_from_x_net('c6c62cc7-fe02-4b18-8f3e-813abfbb4647')
bituach_net['SystemName'] = 'ביטוח'
gemel_net = read_from_x_net('a30dcbea-a1d2-482c-ae29-8f781f5025fb')
gemel_net['SystemName'] = 'גמל'

x_net = pd.concat([pensia_net, bituach_net, gemel_net])

In [ ]:
# leave only relevant data
first_month_of_q = (int(quarter[-1:])) * 3 - 2
last_month_of_q = first_month_of_q + 2
year = quarter[0:4]
# yearmonth = year + '0' + str(first_month_of_q)
yearmonth = year + '0' + str(last_month_of_q)

# x_net_net_first_month_of_q = x_net[x_net['REPORT_PERIOD'] == yearmonth]
x_net_net_last_month_of_q = x_net[x_net['REPORT_PERIOD'] == yearmonth]

funds = pd.merge(left=x_net_net_last_month_of_q,
                 right=funds_agg_by_product,
                 how='outer',
                 left_on=['MANAGING_CORPORATION_LEGAL_ID', 'FUND_ID'],
                 right_on=['ParentCorpLegalId', "ProductNum"],
                 suffixes=['_x_net', '']
                )
funds["SystemName"] = funds["SystemName"].fillna(funds["SystemName_x_net"])
funds.drop("SystemName_x_net", axis=1, inplace=True)
pd.crosstab(funds["SystemName"], funds["report_id"].notnull())

In [ ]:
missing_funds = funds[(funds["report_id"].isnull()) | (funds["FUND_ID"].isnull())]
missing_funds.head()

In [ ]:
# aggregate x_net data per company
funds.groupby(["MANAGING_CORPORATION_LEGAL_ID", "ParentCorpName", "SystemName"])[["TOTAL_ASSETS", "שווי"]].sum()

<h1><center>*** Add fossil classification ***</center></h1>
Based on quarterly company level reports classification

## get previous fossil classifications

In [116]:
prev_cls_fn = "data_sources/prev_class.csv"
prev_by_sec_num, prev_by_ISIN = get_latest_fossil_classifications(prev_cls_fn)

/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:615: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_csv = pd.read_csv(prev_cls_fn, parse_dates=['classification_date'])


previously classified Israeli security nums by is_fossil:
0.00    3256
1.00     315
NaN      139
Name: is_fossil, dtype: int64
previously classified ISINs by is_fossil:
0.00    7781
1.00     686
NaN        1
Name: is_fossil, dtype: int64


## directly using previous company reports
Could be useful for fund level reports, commented out for now

In [117]:
# # TODO: using different file per holding_type for now, should change
# q_company_report1 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות באגח קונצרני.csv')
# q_company_report2 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות במניות.csv')

# q_company_report = pd.concat([
#     q_company_report1[['מספר ני"ע', 'is_fossil']],
#     q_company_report2[['מספר ני"ע', 'is_fossil']]
# ])

# q_company_report = q_company_report.drop_duplicates(subset=['מספר ני"ע']).set_index('מספר ני"ע')
# q_company_report.index = q_company_report.index.astype('str')

## Using previous classification

In [118]:
# don't run when processing fund reports
all_holdings_ever = pd.read_csv(fetch_all_holdings_path(), dtype=str)

In [119]:
all_holdings_cls = add_fossil_classifications(all_holdings_ever, prev_by_sec_num, prev_by_ISIN)

all_holdings: 1090395
having holding number: 920065
without holding number: 170330


/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdings_with_num['מספר ני"ע'] = holdings_with_num['מספר ני"ע'].astype('str').str.strip().str.upper()


Holdings after fossil classification by Israeli security num:
NaN     622716
0.00    260109
1.00     37240
Name: is_fossil, dtype: int64
Holdings after fossil classification by ISIN:
NaN     489014
0.00    384294
1.00     46757
Name: is_fossil, dtype: int64
total fossil sum: 806380544.2287651
holdings count before classification: 1090395
holdings count after classification: 1090395


## Validate Fossil Classification results

In [ ]:
# look for fossil holdings that aren't מניות or אג"ח קונצרני
# all_holdings_cls[
#     (all_holdings_cls["is_fossil"] > 0) &
#     (~all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני']))
# ]

# looking good!
# TODO: might want to try and catch additional fossil holdings of other types - join by all id types

In [ ]:
should_be_cls = all_holdings_cls[all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני'])]
missing_fossil_cls = pd.DataFrame(
    should_be_cls["is_fossil"].isnull().groupby([should_be_cls["ParentCorpName"], should_be_cls["ReportPeriodDesc"]]).sum()
)
missing_fossil_cls

## Write classified results to CSV & create another file with added id types

In [120]:
all_holdings_cls.to_csv(reports_path+"all_holdings_cls.csv", index=False)
# update all_holdings_cls_all_ids if needed
all_holdings_cls_all_ids = load_mappings_and_add_ids_to_holdings(all_holdings_cls)
all_holdings_cls_all_ids.to_csv(reports_path+"all_holdings_cls_all_ids.csv", index=False)

/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cnt = sum(df[col].astype(str).str.strip().str.contains(pattern, na=False))



Holding file ISIN col is: מספר ני"ע
number of ISINs: 166788 out of 1090395 rows

Holding file מספר תאגיד col is: מספר מנפיק
number of מספר תאגידs: 352367 out of 1090395 rows

no LEIs in holdings file


/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:169: FutureWarning: The default value of regex will change from True to False in a future version.
  df[id_cols[id_type]] = df[id_cols[id_type]].str.replace(id_col_patterns(id_type), "")
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:171: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_type = df[id_type].str.contains(id_col_patterns(id_type), na=False)
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:169: FutureWarning: The default value of regex will change from True to False in a future version.
  df[id_cols[id_type]] = df[id_cols[id_type]].str.replace(id_col_patterns(id_type), "")
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:171: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the grou

מספר ני"עs with matching ISIN: 414029 out of total relevant rows: 918806
מספר תאגידs with matching מספר מנפיק: 351858 out of total relevant rows: 352367
מספר ני"עs with matching מספר מנפיק: 373320 out of total relevant rows: 918806
ISINs with matching מספר מנפיק: 376322 out of total relevant rows: 414029
ISINs with matching LEI: 88971 out of total relevant rows: 414029


In [ ]:
# additional output formats
all_holdings_cls_all_ids.to_pickle(reports_path+"all_holdings_cls_all_ids.pkl", compression='gzip')
all_holdings_cls_all_ids.to_json(reports_path+"all_holdings_cls_all_ids.json", compression='gzip')

# -- End of part II - go to Analysis notebook --

## Get Fund stats

In [ ]:
all_holdings_cls_all_ids["שווי"] = all_holdings_cls_all_ids["שווי"].astype(float)

In [ ]:
fund_stats = get_summary(all_holdings_cls_all_ids, 'ParentCorpName', 'SystemName', 'ProductNum','ShortName')
fund_stats.sort_values('שווי פוסילי', ascending=False).head()

In [ ]:
fund_stats.to_csv(reports_path+"fund_stats.csv", index=False)

In [ ]:
# get fossil holdings per fund

fund_stats[fund_stats["ProductNum"].duplicated()]
# TODO: there is 1 duplicate, fund number 103 - check it later
def get_fossil_holdings(fund_id):
    fossil_holdings = all_holdings_cls[
        (all_holdings_cls["ProductNum"] == fund_id) &
        (all_holdings_cls["is_fossil"] > 0)
    ]
    return fossil_holdings.sort_values("שווי פוסילי", ascending=False)

get_fossil_holdings(11366)

## חיסכון לכל ילד

In [ ]:
fund_stats = pd.read_csv(reports_path+"fund_stats.csv")

In [ ]:
hly = fund_stats[
    fund_stats["ShortName"].str.contains("ילד", na=False)
].sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים", ascending=False).reset_index()
hly.to_csv(reports_path+"hly_fund_stats.csv", index=False)
hly

In [ ]:
fund_stats.groupby(["ParentCorpName"]).count()

In [ ]:
all_holdings_cls.groupby(["SystemName","ParentCorpName","ProductNum"])["שעור מסך נכסי השקעה"].sum()

## Fund level data - Discrepancies
* הלמן - seems like we looked at the wrong company level report - it was חברת חשמל...
<br>The numbers ~fit the ones in the company level PDF report. The gov site is missing a company level report.

the company name is: הלמן-אלדובי קופות גמל ופנסיה בע"מ


In [ ]:
# missing reports
[r for r in funds_df.index.unique() if r not in all_holdings_final["report_id"].unique()]

# checked 2 missing, both have 0 holdings, all good

In [ ]:
pd.DataFrame(all_holdings_final.groupby(['ParentCorpName', 'SystemName'])["report_id"].nunique())

### discrepancies

* report 2101262: empty sheet "זכויות מקרקעין" - explains all the diff
* report 2101276: all holding_types sums are ok, the sum @summary sheet is slightly different for some reason

In [ ]:
print(funds_df[funds_df["ProductNum"] == 2207])
totals[totals["report_id"] == '2101262']

# looking for lost insurance funds

### Migdal

In [ ]:
migdal_ins = all_holdings_final[(all_holdings_final["ParentCorpLegalId"] == '520004896')]
migdal_by_fund = pd.DataFrame(migdal_ins.groupby(["ShortName", "ProductNum", "report_id"], dropna=False)["שווי"].agg('sum'))
migdal_by_fund.reset_index()

https://www.migdal.co.il/about/investments/assets - there are 4 more funds not included in the reports site:
* מגדל משתתף ברווחים - קרן ח - b010_p221_520004896
* מגדל משתתף ברווחים - קרן ט - 520004896_b011_p221
* מגדל משתתף ברווחים - קרן י - b012_p221_520004896
* מגדל משתתף קרן י החדשה - b013_p221_520004896

Manually downloaded, added to the directory.

In [ ]:
pd.DataFrame(all_holdings_final[
    (all_holdings_final["ParentCorpLegalId"].isnull()) |
    (all_holdings_final["ParentCorpLegalId"] == 520004896)
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

### Menora missing reports

In [ ]:
pd.DataFrame(all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ'
].groupby(["ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)).sort_values("ProductNum").reset_index()

* total 17 funds in website, 2 of which mergerd into others --> 15.
<br><br>
<u>missing reports:</u>
1. 18011
1. 18012

download from here: https://www.menoramivt.co.il/rep-list/about-insurance/participating-portfolio-info/insurer-assets

In [ ]:
# adding sums of 2 migrated funds (data for these 2 is 1 quarter behind)
99576.34 + 531016.24 + all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ']["שווי"].sum()

Conclusion: there seems to be missing reports for Menora. Check again next quarter.

### הפניקס ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520023185'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 30 funds
* added 2 missing reports: 7012, 7011 (קרן ט, קרן י)
<br>
https://www.fnx.co.il/investors-relations-hebrew/investment-information-reports/reshimat-nichsey-hamevateah

In [ ]:
url = "https://cdn.menoramivt.co.il/public/docs/20211205/520042540_b18012_p321.xlsx"
filename = "520042540_b18012_p321.xlsx"
ur.urlretrieve(url, filename)

### הראל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520004078'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 34 funds.<br><br>
<B> Manually added missing reports: 259010 259011 259012 259013 </B>
from website: https://www.harel-group.co.il/about/harel-group/harel/mesthtefet/Pages/list-of-assets.aspx

### כלל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520024647'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 21 funds
* missing reports: 14011, 14012
<br><br>
downloaded from their site: https://www.clalbit.co.il/aboutclalinsurance/generalreports/

## הלמן - פנסיה וגמל
?גמל - הבדל בסכום, האם כתוצאה מדוחות חסרים
<br>
₪16,136,063 בדוח מסכם
16,132,489.84 לפי סכום דוחות מסלול

הבדלים קטנים, לא מצאתי הסבר.
אולי הוציאו דוחות בשעות שונות של היום?

In [ ]:
16136063 - 16132489.84

In [ ]:
helman = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("הלמן")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
helman_by_report = pd.DataFrame(helman.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
helman_by_report

## מגדל פנסיה

In [ ]:
migdal_p = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("מגדל")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
migdal_p_report = pd.DataFrame(migdal_p.groupby(["report_id", "ProductNum", "ShortName"])["שווי"].agg('sum')).reset_index()
migdal_p_report.merge(totals,
                     on="report_id",
                     how="left")

# migdal_p_report["שווי"].sum()

* All pension funds from website are on file.<br>
<B> last Q there was also a discrepancy (2.1B, now 2.15B) </B>